In [1]:
# Advent of Code 2022
f = open("input.txt", "r")
dataIn = f.readlines()

In [2]:
f = open("test.txt", "r")
dataTest = f.readlines()

In [ ]:
# DAY 1
data = []
currentSum = 0
for i in range(len(dataIn)):
	s = dataIn[i].strip() # Enlever le \n
	if len(s) > 0:
		currentSum += int(s)
	else:
		data.append(currentSum)
		currentSum=0

print(max(data))

print('----------------------')

print(sum(sorted(data, reverse=True)[:3]))

In [ ]:
# DAY 2
arr1 = ["A", "B", "C"]
arr2 = ["X", "Y", "Z"]
points = lambda shape : arr2.index(shape)+1
def win(opponentShape, shape) : 
	if arr1.index(opponentShape) == arr2.index(shape): # Draw
		return 3

	if (opponentShape == "A" and shape=="Y") or (opponentShape == "B" and shape=="Z") or (opponentShape == "C" and shape=="X"):
		return 6
	
	return 0

total = 0
for row in dataIn:
	opponentShape, shape = row.strip().split(" ")
	winner = win(opponentShape, shape)
	total+=win(opponentShape, shape) + points(shape)
print(total) # 12740

print('----------------------')
total = 0
for row in dataIn:
	opponentShape, instruction = row.strip().split(" ")

	if instruction=="X": # Lose
		shape = arr2[(arr1.index(opponentShape)-1)%3] # Shape d'avant dans le tableau
	elif instruction=="Y": # Draw
		shape = arr2[arr1.index(opponentShape)]
	else: # Win
		shape = shape = arr2[(arr1.index(opponentShape)+1)%3]
	winner = win(opponentShape, shape)
	total+=win(opponentShape, shape) + points(shape)

print(total) # 11980


In [ ]:
# DAY 3
def priority(letter:str):
	alphabet = "abcdefghijklmnopqrstuvwxyz"
	if letter in alphabet:
		return alphabet.index(letter)+1
	return priority(letter.lower()) + 26

total = 0
for row in dataIn:
	first = row[:(len(row)//2)]
	second = row[(len(row)//2):]

	for s in first:
		if s in second:
			total += priority(s)
			break
print(total) # 8394

print('----------------------')
total = 0

for i in range(len(dataIn)//3):
	group = dataIn[3*i:3*i+3]
	for s in group[0]:
		if (s in group[1]) and (s in group[2]):
			total += priority(s)
			break
print(total) # 2413

In [ ]:
# DAY 4
count = 0

for row in dataIn:
	first, second = row.strip().split(",")
	down1, up1 = map(int, first.split("-"))
	down2, up2 = map(int, second.split("-"))
	if (down1<=down2 and up2<=up1) or (down2<=down1 and up1<=up2):
		count+=1

print(count) # 445

print('----------------------')
count = 0
# dataIn = ["2-4,6-8",
# "2-3,4-5",
# "5-7,7-9",
# "2-8,3-7",
# "6-6,4-6",
# "2-6,4-8",]

for row in dataIn:
	first, second = row.strip().split(",")
	down1, up1 = map(int, first.split("-"))
	down2, up2 = map(int, second.split("-"))
	if (up1 >= down2 and down1 <= up2):
		count+=1
print(count) # 445

In [ ]:
# DAY 5
import re
# Récupérer les données initiales des stacks :
n = dataIn.index("\n")
stacksSrc = list(map(lambda row : [row[4*i+1: 4*i+2] for i in range(len(row)//4)], dataIn[:n-1])) # Changer les chaînes de caractères en tableau de lettres uniques

stacks = [] # stacks[i] contient une liste qui représente la (i+1)-ème stack
for i in range(len(stacksSrc[0])):
	stacks.append([])
	for j in range(len(stacksSrc)-1, -1, -1):
		if stacksSrc[j][i] != ' ':
			stacks[i].append(stacksSrc[j][i]) # 'Transposer'


def move(number, i1, i2):
	# Second part
	stacks[int(i2)-1] += stacks[int(i1)-1][-int(number):] # Ajouter les number dernières caisses
	stacks[int(i1)-1] = stacks[int(i1)-1][:-int(number)]


	# First part
	# for _ in range(int(number)):
		# stacks[int(i2)-1].append(stacks[int(i1)-1].pop())

instructions = list(map(lambda s: re.findall("[0-9]+", s), dataIn[n+1:]))

for instr in instructions:
	move(instr[0], instr[1], instr[2])

s = ""
for stack in stacks:
	s+=stack.pop() if len(stack)>0 else ""
print(s)

In [ ]:
# DAY 6
s = dataIn[0]

for i in range(3,len(s)):
	letters = {s[i-3], s[i-2], s[i-1], s[i]}
	if len(letters) == 4:
		print(i+1) # Le i-ème caractère en comptant à partir de 1 --> 1538
		break

for i in range(14,len(s)):
	letters = {s[i-j] for j in range(14)}
	if len(letters) == 14:
		print(i+1) # Le i-ème caractère en comptant à partir de 1 --> 2315
		break


In [ ]:
# DAY 7
root = {
	'/':{
		"__parent": None,
		"__size": 0
		}
}
current_dir = root["/"]

def cd(dir):
	global current_dir
	if dir == "/" :
		current_dir = root['/']
	elif dir == ".." :
		current_dir = current_dir["__parent"]
	else :
		current_dir = current_dir[dir]

# Créer l'arborescence de fichiers :
for row in dataIn:
	instr = row.strip() # Pour enlever les \n
	if instr.startswith("$"):
		if "cd" in instr: cd(instr[5:])
		# Rien à faire pour ls

	elif instr.startswith("dir"): # Folder
		_, name = instr.split(" ")
		current_dir[name] = { "__parent": current_dir }
	else: # File
		size, name = instr.split(" ")
		current_dir[name] = int(size)

# Calculer les poids des dossiers et sauvegarder ceux <= 100_000
total_weight = 0
def weight(dir):
	if type(dir) == int:
		global total_weight
		total_weight+=dir
		return dir
	else:
		s = 0
		for entry in dir:
			if entry.startswith("__"): continue
			s+=weight(dir[entry])

		# if s <= 100000: # First part
		# 	global total_weight
		# 	total_weight+=s
		dir["__size"] = s
		return s

weight(root["/"])

print(total_weight)
print('----------------------')

weightToDelete = 30000000 - (70000000 - total_weight)
min_weight = total_weight
# Recherche du dossier de poids minimum (et >= weightToDelete)
def find_min(dir):
	global min_weight
	size = dir["__size"]
	if size>=weightToDelete and size< min_weight:
		min_weight = size
	for entry in dir:
		# Cas de base : dossier sans sous-dossiers
		if entry.startswith("__") or type(dir[entry])==int: continue
		find_min(dir[entry])

find_min(root["/"])

print(min_weight)


In [ ]:
# DAY 8
# dataIn = ["30373",
# "25512",
# "65332",
# "33549",
# "35390",]
def for_all(f, list):
	for el in list:
		if not f(el): return False
	return True

n = len(dataIn)
cols = [[dataIn[i][j] for i in range(n)] for j in range(n)] # col[j] contient la j-ième colonne de dataIn

visibles = 2*n + 2*(n-2) # Edges
for i in range(1, n-1):
	row = list(dataIn[i])
	for j in range(1, n-1):
		f = lambda el: el < row[j]
		# Vérifier f pour les parties gauche/droite de la ligne et les parties haut/bas de la colonne
		if for_all(f, row[:j]) or for_all(f, row[j+1:]) or for_all(f, cols[j][:i]) or for_all(f, cols[j][i+1:]):
			visibles+=1

print(visibles) # 1849
print('----------------------')

def get_index(f, lst): # Retourne le rang du 1er élément à partir duquel f(el) == False
	for i in range(len(lst)):
		if not f(lst[i]): return (i+1)
	return len(lst)

def get_score(i, j):
	if i==0 or j==0 or i==(n-1) or j==(n-1): return 0

	row = list(dataIn[i])
	col = cols[j]
	rowLeft, rowRight = row[:j], row[j+1:]
	colUp, colDown = col[:i], col[i+1:]
	f = lambda el: el < row[j]
	
	rowLeft.reverse() # Doit être parcouru dans l'autre sens
	colUp.reverse()
	return get_index(f, rowLeft) * get_index(f, rowRight) * get_index(f, colUp) * get_index(f, colDown)

def get_score2(i, j):
	if i==0 or j==0 or i==(n-1) or j==(n-1): return 0

	directions = [(-1,0),(0,1),(1,0),(0,-1)] # Up, Right, Down, Left
	score = 1

	for (di, dj) in directions:
		dist = 1
		ii = i + di
		jj = j + dj
		while 0<=ii<n and 0<=jj<n and dataIn[ii][jj]<dataIn[i][j]:
			dist+=1
			ii+=di
			jj+=dj
		score*=(dist) if 0<=ii<n and 0<=jj<n else dist-1 # Enlever 1 si on a atteint le bord
	return score


all_scores = [[get_score(i, j) for j in range(n)] for i in range(n)]
all_scores2 = [[get_score2(i, j) for j in range(n)] for i in range(n)]
print(max(map(max, all_scores)))  #205632
print(max(map(max, all_scores2)))  #205632

In [ ]:
# DAY 9
# dataIn = ["R 5",
# "U 8",
# "L 8",
# "D 3",
# "R 17",
# "D 10",
# "L 25",
# "U 20",
# ]

movements = {
	"R" : (1, 0),
	"D" : (0, -1),
	"L" : (-1, 0),
	"U" : (0, 1),
}

def add_vect(U, V):
	x1, y1 = U
	x2, y2 = V
	return (x1+x2, y1+y2)

def calc_new_pos(posI, posF):
	xI, yI = posI
	xF, yF = posF

	dx = (xF - xI)
	dy = (yF - yI)
	if abs(dx)<=1 and abs(dy)<=1: # Si ils sont à une distance plus petite que 1 : on ne fait rien
		return posI
	
	if yI==yF: return (xI + dx // abs(dx), yI)
	if xI==xF: return (xI, yI + dy // abs(dy))
	return (xI + dx // abs(dx), yI + dy // abs(dy))


positions = {(0, 0)}
knots = [(0, 0)]*10
for row in dataIn:
	direction, d = row.split(" ")
	move = movements[direction]

	for _ in range(int(d)): # Répeter le movement d fois
		knots[0] = add_vect(knots[0], move) # Head

		for i in range(1, len(knots)): # Pour chaque autre noeud
			knots[i] = calc_new_pos(knots[i], knots[i-1])

		positions.add(knots[-1]) # Tail position

print(len(positions)) # 6026 / 2273

In [ ]:
# DAY 10

cycle = 0
X = 1
signal = 0

def incr_cycle():
	global signal, cycle, draw, i

	# PART 2
	if cycle % 40 in (X - 1, X, X + 1):
		print("#", end="")
	else:
		print(".", end="")
	
	cycle+=1

	if cycle % 40 == 0:
		print()

	# PART 1
	if cycle%40 == 20:
		signal += cycle * X
	

for row in dataIn:
	if row.startswith("noop"): incr_cycle()
	else:
		_, V = row.strip().split(" ")
		incr_cycle()
		incr_cycle()
		X += int(V)
print(signal) # 13520

# PGPHBEAB

In [ ]:
# DAY 11

class Monkey:
	def __init__(self, data, n) -> None:
		self.n = n
		self.count = 0
		self.items = list(map(int, data[1].split(": ")[1].split(", "))) # Initial items of monkey
		self.f = eval("lambda old:" + data[2].split(" = ")[1]) # Operation function

		# Test function
		self.mod = int(data[3].split("divisible by ")[1])
		ifTrue = int(data[4].split("throw to monkey ")[1])
		ifFalse = int(data[5].split("throw to monkey ")[1])
		self.t = eval(f"lambda n: {ifTrue} if n%{self.mod}==0 else {ifFalse}")
	
	def __str__(self) -> str:
		# {', '.join(str(i) for i in self.items)}
		return f"Monkey {self.n} inspected {self.count} items"
	
	def getItem(self, item):
		self.items.append(item)
	def operation(self, value) -> int:
		return self.f(value)
	def test(self, value) -> int:
		return self.t(value)

	def inspectItems(self, monkeys: list):
		while len(self.items):
			item = self.items.pop(0)
			worryLevel = self.operation(item)
			# worryLevel //= 3 # PART 1
			worryLevel %= allDivs # PART 2
			nextMonkey = self.test(worryLevel)

			monkeys[nextMonkey].getItem(worryLevel)
			self.count+=1

# Récupérer les infos
monkeys: list[Monkey] = []
allDivs = 1
for i in range((len(dataIn)+1)//7):
	data = (list(map(lambda s:s.strip(), dataIn[7*i: 7*i+6])))
	newMonkey = Monkey(data, i)
	allDivs *= newMonkey.mod
	monkeys.append(newMonkey)
	


for _ in range(10000):
	for monkey in monkeys:
		monkey.inspectItems(monkeys)
	
sortedCounts = sorted(map(lambda m: m.count, monkeys), reverse=True)
print(sortedCounts[0] * sortedCounts[1]) # 95472 / 17926061332

In [ ]:
# DAY 12
from collections import deque
heightMap = []
Si = Sj = Ei = Ej = 0

for i in range(len(dataIn)):
	newRow = []
	row = dataIn[i].strip()
	for j in range(len(row)): # -1 pour enlever le '\n'
		c = row[j]

		if c == 'S': 
			Si, Sj = (i, j)
			newRow.append(ord('a'))
		elif c == 'E': 
			Ei, Ej = (i, j)
			newRow.append(ord('z'))
		else: newRow.append(ord(c))

	heightMap.append(newRow)

# Parcours en largeur : depuis E vers S (plus simple pour la partie 2)
q = deque([(0, Ei, Ej)]) # Distance, i, j
visited = {(Ei, Ej)}
n, p = len(heightMap), len(heightMap[0])

while len(q):
	d, i, j = q.popleft()
	for ii, jj in [(i+1, j), (i-1, j), (i, j+1), (i, j-1)]:
		# Cas impossibles : hors de la grille, déjà visité, trop bas
		if not (0 <= ii < n and 0 <= jj < p) or (ii, jj) in visited or heightMap[ii][jj] < heightMap[i][j] -1 :
			continue
		
		# if (ii, jj) == (Si, Sj): # PART 1
		if heightMap[ii][jj] == ord('a'): # PART 2
			print("Distance :", d+1)
			q = []
			break
		
		visited.add((ii, jj))
		q.append((d+1, ii, jj))



In [ ]:
# DAY 13
def compare(l, r): # Retourne l - r (au sens large)
	if type(l)==int and type(r)==int:
		return l-r
	
	if type(l)==list and type(r)==list:
		if len(l)==0 or len(r)==0: return len(l) - len(r)
		
		c = compare(l[0], r[0])
		return c if c != 0 else compare(l[1:], r[1:])
		
	
	if type(l)==int: return compare([l], r)
	if type(r)==int: return compare(l, [r])


s = 0
data2 = []
for i in range((len(dataIn)+3)//3):
	left, right = dataIn[3*i: 3*i+2]
	left = eval(left)
	right = eval(right)

	if compare(left, right) < 0:
		s += i+1
	
	# PART 2
	data2.append(left)
	data2.append(right)

print(s) # 6428

# PART 2
from functools import cmp_to_key

data2.append([[2]])
data2.append([[6]])
data2 = sorted(data2, key=cmp_to_key(compare))

print((data2.index([[2]]) + 1) * (data2.index([[6]]) + 1)) # 22464



In [ ]:
# DAY 14

rocks, sands = set(), set()
Hmin = 0
Xmin, Xmax = float("inf"), -float("inf")

def display_cave():
    for y in range(0, Hmin+1):
        for x in range(Xmin-6, Xmax+7):
            if (x, y) in sands:
                print('o', end="")
            elif (x, y) in rocks:
                print('#', end="")
            else:
                print(".", end="")
        print("")

# Créer la grotte
for row in dataIn:
	L = row.strip().split(" -> ")
	for V1, V2 in zip(L, L[1:]): # zip(L, L[1:]) renvoie les couples (L[0],L[1]), (L[1],L[2])...
		x1, y1 = map(int, V1.split(","))
		x2, y2 = map(int, V2.split(","))

		# Une des 2 boucles ne fait qu'un tour car x1==x2 ou y1==y2
		for x in range(min(x1, x2), max(x1, x2)+1):
			for y in range(min(y1, y2), max(y1, y2)+1):
				rocks.add((x, y))
				Hmin = max(Hmin, y) # Un max car y va en descendant (0, 1...Hmin)
				Xmin, Xmax = min(Xmin, x), max(Xmax, x)


Hmin+=2

# PART 2
for x in range(Xmin-Hmin-10, Xmax+Hmin+10):
	rocks.add((x, Hmin))
	

def down(x, y): return (x, y+1)
def left(x, y): return (x-1, y+1)
def right(x, y): return (x+1, y+1)

def add_sand(x, y):
	#Si on dépasse les derniers rochers : Part 1
	# if y>Hmin-2:
	# 	return 0

	for f in [down, left, right]:
		i, j = f(x,y)
		# Si la case est libre
		if not((i, j) in rocks or (i, j) in sands):
			return add_sand(i, j)
	sands.add((x, y))
	return 1
	

while (500, 0) not in sands:
	a = add_sand(500, 0)

	# Si le sable tombe à l'infini
	if a==0:
		break

print(len(sands)) # 698 / 28594


In [ ]:
# DAY 15
from re import findall

S = set()
# get all sensors
for row in dataIn:
	xs, ys, xb, yb = map(int, findall("-?[0-9]+", row))
	d = abs(xs-xb) + abs(ys-yb)

	S.add((xs, ys, xb, yb, d))


def get_occupied(Y, Xmin=None, Xmax=None):
	intervals = []
	# Récupérer les intervalles occupés de la ligne
	for xs, ys, xb, yb, d in S:
		delta = abs(Y - ys)
		if delta <= d:
			intervals.append((xs - (d - delta), xs + (d - delta) + 1))

	intervals.sort() # Intervalles qui contiennent les cases occupées
	lastb = intervals[0][0] - 1 # Plus petite borne
	ranges = []

	# Fusionner tous les intervalles (le résultat peut être une réunion d'intervalles)
	for a, b in intervals:
		# Restreindre l'intervalle à [Xmin, Xmax]
		if Xmin is not None and Xmax is not None:
			a = max(a, Xmin)
			a = min(a, Xmax)
			b = max(b, Xmin)
			b = min(b, Xmax)

		if a >= b or b<=lastb:
			continue

		# Les intervalles sont triés dans l'odre croissant : il suffit juste de les prolonger à droite (en b)
		if a <= lastb:
			ranges[-1][1] = b
		else: # Cas où le nouvel intervalle est disjoint des précédents
			ranges.append([a, b])
		lastb = b
	
	s = sum(b-a for a,b in ranges) # Longueur des intervalles = nombre de cases occupées
	return s, ranges

lineY = 2000000
count, _ = get_occupied(lineY)
count -= len(set((bx,by) for _,_,bx,by,_ in S if by == lineY)) # Enlever les balises B connues
print("PART 1 :", count)

# PART 2:
M = 4_000_000
for Y in range(M+1):
	count, ranges = get_occupied(Y, 0, M+1)
	if count!=M+1: # Si une case n'est pas occupée
		x = ranges[0][1] # 1 seule case est libre, la borne supérieure du 1er intervalle
		print("PART 2 :", x*4_000_000 + Y) # 12480406634249 (~1min)
		break

In [ ]:
# DAY 16
from re import findall
from collections import deque

# Récupérer les données du pb
valves = {}
tunnels = {}
for row in dataIn:
	parsedRow = findall("[A-Z]{2}", row)
	valve, nextValves = parsedRow[0], parsedRow[1:]
	rate = int(findall("[0-9]+", row)[0])
	
	valves[valve] = rate
	tunnels[valve] = nextValves


dists = {}
# Calculer toutes les distances entre des sommets de rate non nul (Parcours en largeur car graphe non pondéré)
# Pour chaque vanne : 
for valve in valves:
	if valve!="AA" and valves[valve]==0 : # Si le rate est nul on ne calcule pas les distances (sauf pour "AA")
		continue

	# Parcours en largeur "classique" (en ignorant les sommets de rate nul)
	q = deque([(valve, 0)]) # Valve, distance
	visited = {valve}

	dists[valve] = {}

	while len(q):
		position, d = q.popleft()
		for neighbor in tunnels[position]:
			if neighbor in visited:
				continue
						
			visited.add(neighbor)
			q.append((neighbor, d+1))
			if valves[neighbor]:
				dists[valve][neighbor] = d+1

cache = {}
# Parcourir toutes les possibilités d'ouverture des vannes:
def find_best_path(start, time, opened_valves = set()): # Valve de départ ; temps restant ; vannes actuellement ouvertes
	max_v = 0

	j = "".join(opened_valves)
	if (start, time, j) in cache:
		return cache[(start, time, j)]

	# Pour chaque autre valve de rate non nul :
	for neighbor in dists[start]:
		if neighbor in opened_valves:
			continue

		time_spent = dists[start][neighbor] + 1 # Temps de parcours + 1min pour ouvrir
		remaining_time = time-time_spent
		if remaining_time <= 0:
			continue

		max_v = max(max_v, find_best_path(neighbor, remaining_time, opened_valves.union({neighbor})) + valves[neighbor] * remaining_time)

	cache[(start, time, j)] = max_v
	return max_v

print(find_best_path("AA", 30)) # 1580

def find_best_path_2(start, time):
	max_v = 0
	n = len(dists)-1
	# On teste toutes les combinaisons possibles où l'homme ouvre i vannes et l'éléphant (n-i)
	# Il suffit de tester pour i<=n/2 car l'homme et l'éléphant jouent des rôles symétriques
	b = 2**n -1
	for i in range((b+1)): # Pour chaque combinaison possible de 10001011: On teste avec les vannes ouvertes correspondantes
		s = str(bin(i))[2:]
		valves_h = set() # Homme
		valves_e = set() # Eléphant
		j = 0
		for v in dists :
			if j<len(s) and s[j]=="1": valves_h.add(v)
			else: valves_e.add(v)
			j+=1
		
		
		max_v = max(max_v, find_best_path("AA", 26, valves_h) + find_best_path("AA", 26, valves_e))

	return max_v


print(find_best_path_2("AA", 26)) # 2213 (3min)

In [ ]:
# DAY 17
moves = [1 if c == '>' else -1 for c in dataIn[0]]
n = len(moves)

all_rocks = set((x, -1) for x in range(7))
Hmax = 0

rock_shapes = [
    [(x, 0) for x in range(4)],
    [(0, 1), (1, 0), (1, 1), (1, 2), (2, 1)],
    [(0, 0), (1, 0), (2, 0), (2, 1), (2, 2)],
    [(0, y) for y in range(4)],
    [(x, y) for x in range(2) for y in range(2)],
]


def place_rock(rock):
    return {(M[0]+2, M[1]+Hmax+3) for M in rock}


def move_rock(rock, m):
    global all_rocks, Hmax
    new_rocks = rock
    moved = rock
    i_jet = m
    while len(moved.intersection(all_rocks)) == 0:  # Tant qu'il n'y a pas de collisions
        new_rocks = moved
        dir = moves[i_jet]
        i_jet = (i_jet+1) % n
        # ----- Déplacement horizontal -----
        # Max si on va à droite et min sinon
        X_m = max(new_rocks, key=lambda M: M[0] * dir)
        moved = {(M[0]+dir, M[1]) for M in new_rocks}

        if (0 <= X_m[0] + dir < 7) and not(len(moved.intersection(all_rocks))):
            new_rocks = moved
        
        # ----- Déplacement vertical -----
        moved = {(M[0], M[1]-1) for M in new_rocks}

    all_rocks = all_rocks.union(new_rocks)
    Hmax = max(Hmax, max(new_rocks, key=lambda M: M[1])[1] + 1)

    return new_rocks, i_jet


i_jet = 0
for i in range(2022):
    rock = place_rock(rock_shapes[i % 5])
    new_rocks, i_jet = move_rock(rock, i_jet)
print(Hmax)


In [ ]:
# DAY 17 - Solution
datpos = 0
dat = dataIn[0].strip()
shapes = [
	[[1,1,1,1]],
	[[0,1,0],[1,1,1],[0,1,0]],
	[[1,1,1],[0,0,1],[0,0,1]],
	[[1],[1],[1],[1]],
	[[1,1],[1,1]],
]

well = set()
wellheight = 0
wellwidth = 7
def doesoverlap(i, x, y):
	for dy in range(len(shapes[i])):
		for dx in range(len(shapes[i][dy])):
			if shapes[i][dy][dx] and (x+dx,y+dy) in well:
				return True
	return False
def drop(i):
	global wellheight, datpos
	x = 2
	y = wellheight + 3
	while True:
		d = dat[datpos]
		datpos = (datpos + 1) % len(dat)

		oldx = x
		if d == '<':
			x -= 1
		else:
			x += 1
		if x < 0 or x + len(shapes[i][0]) > wellwidth or doesoverlap(i, x, y):
			x = oldx

		y -= 1
		if y < 0 or doesoverlap(i, x, y):
			y += 1
			break

	for dy in range(len(shapes[i])):
		for dx in range(len(shapes[i][dy])):
			if shapes[i][dy][dx]:
				well.add((x + dx, y + dy))
				if y + dy + 1 > wellheight:
					wellheight = y + dy + 1

for i in range(2022):
	drop(i % len(shapes))
print(wellheight)

well = set()
wellheight = 0
datpos = 0

def getwellstate(i):
	a = []
	for x in range(wellwidth):
		y = wellheight
		while (x,y) not in well and y >= 0:
			y -= 1
		a.append(wellheight - y)
	a.append(datpos)
	a.append(i)
	return tuple(a)

states = {}
wellheights = [0]
i = 0
while True:
	drop(i % len(shapes))
	state = getwellstate(i % len(shapes))
	i += 1
	wellheights.append(wellheight)
	if state in states:
		oldi, oldwellheight = states[state]
		n = 1000000000000 - oldi
		loops, rest = divmod(n, i - oldi)
		res = wellheights[oldi + rest] + loops * (wellheight - oldwellheight)
		print(res)
		break
	else:
		states[state] = i, wellheight

In [ ]:
# DAY 18
from collections import deque

faces = {}
cubes = set()
offsets = [(0, 0, 0.5), (0, 0, -0.5), (0, 0.5, 0), (0, -0.5, 0), (0.5, 0, 0), (-0.5, 0, 0)]

Xmin = Ymin = Zmin = float("inf")
Xmax = Ymax = Zmax = -float("inf")

for row in dataIn:
	x, y, z = map(int, row.split(","))
	cubes.add((x,y,z))
	Xmin, Ymin, Zmin = min(Xmin, x), min(Ymin, y), min(Zmin, z)
	Xmax, Ymax, Zmax = max(Xmax, x), max(Ymax, y), max(Zmax, z)

	for dx, dy, dz in offsets:
		F = (x+dx, y+dy, z+dz) # Représente un face donc coordonnées demi-entières
		if F not in faces:
			faces[F] = 0
		faces[F]+=1

print("Part 1 :", sum(filter(lambda k: k==1, faces.values()))) # Compter le nb de 1 --> 3662

Xmin -= 1
Ymin -= 1
Zmin -= 1

Xmax += 1
Ymax += 1
Zmax += 1

air = {(Xmin, Ymin, Zmin)} # Ensembles des cases qui contiennent de l'air
q = deque([(Xmin, Ymin, Zmin)]) # File pour parcours en largeur

# Parcours des cases d'air pour créer l'"enveloppe" extérieure
while len(q):
	x, y, z = q.popleft()

	for dx, dy, dz in offsets:
		M = (x+2*dx, y+2*dy, z+2*dz) # Cases adjacentes --> +2*dx et pas +dx

		if (not (Xmin<= M[0] <= Xmax and Ymin<= M[1] <= Ymax and Zmin<= M[2] <= Zmax)) or M in cubes or M in air:
			continue

		air.add(M)
		q.append(M)

# Les faces en contact avec l'extérieur sont dans 'faces' et sont des faces de points de 'air'
facesAir = set() # faces des cubes d'air extérieurs
for x,y,z in air:
	for dx, dy, dz in offsets:
		facesAir.add((x+dx, y+dy, z+dz))

print("Part 2 :", len(facesAir.intersection(faces))) # 2060


In [24]:
# DAY 19
from re import findall

def dfs_geodes(costs: dict, time: int, bots: dict, resources: dict, cache: dict):
	if time==0: return resources["geode"]
	
	# Le nb de geodes est le même pour time, bots et resources identiques --> mise en cache
	key = (time, str(bots), str(resources))
	if key in cache:
		return cache[key]

	max_v = resources["geode"] + bots["geode"]*time  # Valeur minimum : nb de geodes + Nombre actuel de geodebot * tps restant
	
	# Pour chaque bot
	for botName, cost in costs.items():
		# Si on ne produit pas une des resources nécéssaires :
		toStop = False
		for res, val in cost.items():
			if bots[res]==0: # Si il n'y a pas de bot qui produit cette ressource
				toStop = True
				break
		if toStop: break
		# Si on ne peux pas le construire avec les ressources dispo :
		for cost, value in costs[bot]:
			if value > resources

		time_spent = dists[start][neighbor] + 1 # Temps de parcours + 1min pour ouvrir
		remaining_time = time-time_spent
		if remaining_time <= 0:
			continue

		max_v = max(max_v, dfs_geodes(costs, remaining_time, bots2, resources2, cache ))

	
	cache[key] = max_v
	return max_v

s=0
for row in dataTest:
	id, orebot_ore, claybot_ore, obsibot_ore, obsibot_clay, geodebot_ore, geodebot_obsi = findall("\d+", row)
	costs = {
		"ore":{ "ore" : orebot_ore },
		"clay":{ "ore" : claybot_ore },
		"obsi":{ "ore" : obsibot_ore, "clay": obsibot_clay },
		"geode":{ "ore" : geodebot_ore, "obsi": geodebot_obsi },
	}

	bots = { "ore": 1, "clay": 0, "obsi": 0, "geode": 0 }
	resources = { "ore": 0, "clay": 0, "obsi": 0, "geode": 0 }
	max_g = dfs_geodes(costs, 24, bots, resources, {})
	s += id*max_g

print(s)

for k, c in costs.items():
	print(k + " :", c)

0
ore : {'ore': '2'}
clay : {'ore': '3'}
obsi : {'ore': '3', 'clay': '8'}
geo : {'ore': '3', 'obsi': '12'}


In [38]:
bots = { "ore": 1, "clay": 0, "obsi": 0, "geode": 0 }
"".join(map(str,bots.values()))


True

In [23]:
line = dataTest[1]
bp = []
maxspend = [0, 0, 0]
for section in line.split(": ")[1].split(". "):
	recipe = []
	for x, y in findall(r"(\d+) (\w+)", section):
		x = int(x)
		y = ["ore", "clay", "obsidian"].index(y)
		recipe.append((x, y))
		maxspend[y] = max(maxspend[y], x)
	bp.append(recipe)
v = dfs(bp, maxspend, {}, 24, [1, 0, 0, 0], [0, 0, 0, 0])
total += (i + 1) * v
bp

[[(2, 0)], [(3, 0)], [(3, 0), (8, 1)], [(3, 0), (12, 2)]]

In [25]:
def dfs(bp, maxspend, cache, time, bots, amt):
    if time == 0:
        return amt[3]
    
    key = tuple([time, *bots, *amt])
    if key in cache:
        return cache[key]
    
    maxval = amt[3] + bots[3] * time
    
    for btype, recipe in enumerate(bp):
        if btype != 3 and bots[btype] >= maxspend[btype]:
            continue
    
        wait = 0
        for ramt, rtype in recipe:
            if bots[rtype] == 0:
                break
            wait = max(wait, -(-(ramt - amt[rtype]) // bots[rtype]))
        else:
            remtime = time - wait - 1
            if remtime <= 0:
                continue
            bots_ = bots[:]
            amt_ = [x + y * (wait + 1) for x, y in zip(amt, bots)]
            for ramt, rtype in recipe:
                amt_[rtype] -= ramt
            bots_[btype] += 1
            for i in range(3):
                amt_[i] = min(amt_[i], maxspend[i] * remtime)
            maxval = max(maxval, dfs(bp, maxspend, cache, remtime, bots_, amt_))
    
    cache[key] = maxval
    return maxval